# **Import Libraries**

In [7]:
# Step 1: Import necessary libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# **Mount Google Drive and Define Paths and Create Folders**

In [8]:
from google.colab import drive
drive.mount('/content/drive')
# Define Paths
input_folder = "/content/drive/MyDrive/Dataset/train/train_data"
csv_path = "/content/drive/MyDrive/Dataset/train/other/train.csv"
output_folder = "/content/drive/MyDrive/Dataset/train/EfficientNetB0/augmented_img"
models_folder = "/content/drive/MyDrive/Dataset/train/EfficientNetB0/models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Load CSV File and Add Class Names**

In [10]:
# Load Dataset
df = pd.read_csv(csv_path)

# Define label mapping
label_mapping = {
    0: "Cassava Bacterial Blight (CBB)",
    1: "Cassava Brown Streak Disease (CBSD)",
    2: "Cassava Green Mottle (CGM)",
    3: "Cassava Mosaic Disease (CMD)",
    4: "Healthy"
}

# Map labels to class names
df['class_name'] = df['label'].map(label_mapping)

# Show a message confirming the full execution
print("Dataset loaded and class names mapped successfully!")
print("First 5 rows of the dataset:")
print(df.head())

Dataset loaded and class names mapped successfully!
First 5 rows of the dataset:
         image_id  label                           class_name
0  1000015157.jpg      0       Cassava Bacterial Blight (CBB)
1  1000201771.jpg      3         Cassava Mosaic Disease (CMD)
2   100042118.jpg      1  Cassava Brown Streak Disease (CBSD)
3  1000723321.jpg      1  Cassava Brown Streak Disease (CBSD)
4  1000812911.jpg      3         Cassava Mosaic Disease (CMD)


# **Split Dataset into Training and Validation Sets**

In [11]:
# Train-Validation Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Print a message confirming the split
print(f"Dataset split into training and validation sets: {len(train_df)} training samples and {len(val_df)} validation samples.")


Dataset split into training and validation sets: 17117 training samples and 4280 validation samples.


# **Define Image Dimensions and Data Augmentation**

In [12]:
# Image Parameters
img_height, img_width = 224, 224
batch_size = 32

# **Create Data Generators**

In [13]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
# Data Generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=input_folder,
    x_col='image_id',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw'
)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=input_folder,
    x_col='image_id',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

Found 13484 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3633 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


Found 3414 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 866 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


# **Visualize and Save Augmented Images**

In [ ]:
# Function to visualize and save augmented images with corresponding augmentation types in filenames
def visualize_and_save_augmented_images(generator, save_dir, num_images=30):
    # Get the first batch of images and labels
    images, labels = next(generator)

    # Get the original filenames from the generator
    filenames = generator.filenames

    # Define augmentation types and their filenames
    augmentations = [
        {'rescale': 1./255, 'filename': 'rescale_1.0_255.jpg'},
        {'rotation_range': 40, 'filename': 'rotation_range_40.jpg'},
        {'width_shift_range': 0.3, 'filename': 'width_shift_range_0.3.jpg'},
        {'height_shift_range': 0.3, 'filename': 'height_shift_range_0.3.jpg'},
        {'shear_range': 0.3, 'filename': 'shear_range_0.3.jpg'},
        {'zoom_range': 0.3, 'filename': 'zoom_range_0.3.jpg'},
        {'horizontal_flip': True, 'filename': 'horizontal_flip_True.jpg'},
        {'vertical_flip': True, 'filename': 'vertical_flip_True.jpg'},
        {'brightness_range': [0.8, 1.2], 'filename': 'brightness_range_0.8_1.2.jpg'},
        {'fill_mode': 'nearest', 'filename': 'fill_mode_nearest.jpg'}
    ]

    # Create a subfolder for each image and save the augmented images
    for i in range(num_images):
        # Extract the original image's filename (without extension) and label
        original_filename = os.path.splitext(os.path.basename(filenames[i]))[0]  # Get filename without extension
        label = labels[i]  # Get the label

        # Create a folder name using the original filename and label
        folder_name = f"{original_filename}_label_{label}"
        image_folder = os.path.join(save_dir, folder_name)  # Create folder for each image
        os.makedirs(image_folder, exist_ok=True)

        # Save the original image
        img = array_to_img(images[i])  # Convert array to image
        img.save(os.path.join(image_folder, 'original_image.jpg'))  # Save original image

        # Display the original image
        plt.figure()
        plt.imshow(images[i])
        plt.title(f'Original Image - Label: {label}')
        plt.axis('off')
        plt.show()

        # Apply each augmentation and save the image with corresponding filename
        for aug in augmentations:
            # Create a new ImageDataGenerator with the specific augmentation
            datagen = ImageDataGenerator(**{k: v for k, v in aug.items() if k != 'filename'})
            augmented_image = datagen.random_transform(images[i])  # Apply the augmentation
            augmented_img = array_to_img(augmented_image)  # Convert array to image

            # Save augmented image with the augmentation type in the filename
            augmented_filename = aug['filename']
            augmented_img.save(os.path.join(image_folder, augmented_filename))  # Save augmented image

            # Display the augmented image
            plt.figure()
            plt.imshow(augmented_image)
            plt.title(f'Augmented Image: {augmented_filename} - Label: {label}')
            plt.axis('off')
            plt.show()

# Call the function to visualize and save augmented images
visualize_and_save_augmented_images(train_generator, output_folder)


# **Define and Compile the Model**

In [ ]:
# Step 16: Define the CNN model architecture
num_classes = len(label_mapping)  # Number of unique classes
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),  # First convolutional layer
    layers.MaxPooling2D((2, 2)),  # Max pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
    layers.MaxPooling2D((2, 2)),  # Max pooling layer
    layers.Conv2D(128, (3, 3), activation='relu'),  # Third convolutional layer
    layers.MaxPooling2D((2, 2)),  # Max pooling layer
    layers.Flatten(),  # Flatten the output
    layers.Dense(512, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(num_classes, activation='softmax')  # Output layer with softmax activation
])

# Step 17: Compile the model
model.compile(
    optimizer='adam',  # Optimizer
    loss='sparse_categorical_crossentropy',  # Loss function for integer labels
    metrics=['accuracy']  # Metric to monitor
)

# Step 18: Print the model summary
print("Model Summary:")
model.summary()

# **Define Callbacks and Train the Model**

In [ ]:
# Call function to visualize and save augmented images
visualize_and_save_augmented_images(train_generator, output_folder)

# Compute Class Weights
class_weights = compute_class_weight('balanced', classes=np.unique(df['label']), y=df['label'])
class_weight_dict = dict(enumerate(class_weights))

# Define Model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_mapping), activation='softmax')
])

# **Compile Model**

In [ ]:
# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# **Define Callbacks**

In [ ]:
# Define Callbacks
class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        print(f"Model saved successfully at epoch {epoch + 1} with validation accuracy: {logs.get('val_accuracy'):.4f}")

checkpoint_callback = CustomModelCheckpoint(
    filepath=os.path.join(models_folder, 'model_epoch_{epoch:02d}_val_acc_{val_accuracy:.4f}.keras'),
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# **Train and save The Final Model**

In [ ]:
# Train Model
epochs = 50
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[checkpoint_callback, lr_scheduler, early_stopping]
)
# Save Final Model
final_model_path = os.path.join(models_folder, 'final_disease_classifier.h5')
model.save(final_model_path)
print(f"Final model saved at: {final_model_path}")

# **Prediction Function and Example Usage**

In [ ]:
# Test Data Directory
test_folder = "/content/drive/MyDrive/Dataset/train/test_data"  # Adjust the path to your test data folder

# Test Data Generator
test_datagen = ImageDataGenerator(rescale=1./255)

# Test Generator (adjust according to your test dataset)
test_generator = test_datagen.flow_from_directory(
    directory=test_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,  # We don't have labels in the test set
    shuffle=False  # Important: We don't shuffle the test data
)

# Prediction Function for Batch Prediction
def predict_test_data(generator):
    filenames = generator.filenames
    predictions = model.predict(generator, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)  # Get the predicted class indices
    predicted_labels = [label_mapping[class_idx] for class_idx in predicted_classes]  # Map class index to label

    # Display predictions and save images
    for i, (filename, label) in enumerate(zip(filenames, predicted_labels)):
        image_path = os.path.join(test_folder, filename)
        img = tf.keras.preprocessing.image.load_img(image_path)  # Load the image
        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.title(f"Predicted: {label}")
        plt.axis('off')
        plt.show()

    return filenames, predicted_labels

# Call the function to predict and visualize test images
test_filenames, predicted_labels = predict_test_data(test_generator)

# Print the predicted labels and class names
for filename, label in zip(test_filenames, predicted_labels):
    print(f"Image: {filename} - Predicted Label: {label} - Class Name: {label_mapping[int(label)]}")
